In [ ]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

# Pinecone API credentials and index details
PINECONE_API_KEY = "2bd5fd9f-2c56-42f9-aa1b-037960262fff"
PINECONE_API_ENV = "us-east-1-aws"
INDEX_NAME = "medicalbot"

# Initialize Pinecone
pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV)

# Load data from PDF files
def load_pdf(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents

extracted_data = load_pdf("data/")
print(f"Loaded {len(extracted_data)} documents from PDF files.")

# Split text into manageable chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20
    )
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

text_chunks = text_split(extracted_data)
print("Length of text chunks:", len(text_chunks))

# Download Hugging Face embeddings
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = download_hugging_face_embeddings()

# Connect to existing Pinecone index and add new data if needed
try:
    docsearch = Pinecone.from_existing_index(index_name=INDEX_NAME, embedding=embeddings)
    print("Connected to existing Pinecone index.")
except Exception as e:
    print(f"Failed to connect to existing index: {e}")
    print("Creating a new index and storing data...")
    docsearch = Pinecone.from_texts(
        [t.page_content for t in text_chunks], 
        embeddings, 
        index_name=INDEX_NAME
    )

# Define query for similarity search
query = "What are Allergies?"
docs = docsearch.similarity_search(query, k=3)
print("Retrieved documents:", docs)

# Define the PromptTemplate for the chatbot
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know; don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}

# Initialize the LLM
llm = CTransformers(
    model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
    model_type="llama",
    config={
        'max_new_tokens': 512,
        'temperature': 0.8
    }
)

# Build the RetrievalQA chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

# Query the QA system
result = qa.run(query)
print("Answer:", result)
